In [1]:
#IMPORTING REQUIRED MODULES

import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
#reading required data files

newdf=pd.read_excel(r'DDW-C19-0000.xlsx') #POPULATION BY BILINGUALISM, TRILINGUALISM, EDUCATIONAL LEVEL AND SEX
data=pd.read_excel('DDW-0000C-08.xlsx') #EDUCATIONAL LEVEL BY AGE AND SEX FOR POPULATION AGE 7 AND ABOVE 

In [3]:
#selecting only the required columns

newdf=newdf.iloc[:,[0,2,3,4,8]]

In [4]:
#renaming the columns

newdf.columns=['statecode','state','tru','level','3.more']

In [5]:
#removing the rows that contain metadata

newdf=newdf.iloc[5:-3,:]

In [6]:
#selecting rows that have level as one of the literacy groups

newdf=newdf[newdf['level']!='Total']
newdf=newdf[newdf['level']!='Literate']

#selecting rows that have tru as total

newdf=newdf[newdf['tru']=='Total']

In [7]:
#removing tru column

newdf=newdf.iloc[:,[0,3,4]]

In [8]:
#selecting only the req rows and columns from 'data' dataframe

data=data.iloc[6:,[1,3,4,5,9,15,18,21,24,27,30,33,36,39]]

In [9]:
#renaming the columns

data.columns=['statecode','statename','tru','age','illiterate','we','bepri','pri','mid','sec','highsec','ntd','td','gna']

In [10]:
#selecting rows that have tru as total and age as all ages

data=data[data['tru']=='Total']
data=data[data['age']=='All ages']

In [11]:
#removing tru and age columns

data=data.iloc[:,[0,1,4,5,6,7,8,9,10,11,12,13]]

In [12]:
#combining various literacy clases into one class to make them consistent across files

# combined below primary and literate without education level into below primary

data['bepri']=data['we']+data['bepri']

#combined secondary,higher secondary, non technical diploma and technical diploma into secondary

data['sec']=data['sec']+data['highsec']+data['ntd']+data['td']

#selecting only the required columns

data=data.iloc[:,[0,2,4,5,6,7,11]]

In [13]:
#creating a new dataframe

newdata=pd.DataFrame()

In [14]:
#converting literacy subgroup values which are as columns into rows of new data

for i in range(len(data)):
    for j in range(6):
        newdata=newdata.append(pd.Series([data.iloc[i,0],data.iloc[i,j+1]]),ignore_index=True)
        
#naming the columns of newdata dataframe

newdata.columns=['statecode','pop']

#adding column 'level' to new data to detct which literacy group that row belongs to

newdata['level']='a'
l=['Illiterate','Literate but below primary','Primary but below middle','Middle but below matric/secondary','Matric/Secondary but below graduate','Graduate and above']
count=0
for i in range(len(newdata)):
    newdata.iloc[i,2]=l[count]
    count=(count+1)%6

In [15]:
#merging newdf and newdata

merged=pd.merge(newdf,newdata)

In [16]:
#printing the columns present in merged data frame

merged.columns

Index(['statecode', 'level', '3.more', 'pop'], dtype='object')

In [17]:
#converting string to numeric value

merged['pop']=pd.to_numeric(merged['pop'])
merged['3.more']=pd.to_numeric(merged['3.more'])

In [18]:
#calculating the percentage of three of more lang sepaking people in that age group

merged['percent']=merged['3.more']/merged['pop']

In [19]:
#selecting only the required columns

merged=merged.iloc[:,[0,1,-1]]

In [20]:
#creating a new dataframe to store the literacy group that has max percentage in that state

high=pd.DataFrame()

#adding statecode column to high data frame

g=merged.groupby(by=['statecode'],as_index=False)

#this list is will store the statecodes

statecode_list=[]

for i,j in g:
    statecode_list.append(i)
    
#assigning the statecode_list to statecode column of high dataframe

high['statecode']=statecode_list

#getting the highest percentage value of  literacy group that speaks 3 or more langs

high['percent']=merged.groupby(by=['statecode'],as_index=False).agg(func='max')['percent']

#merging the high and merged data frame to get the finalmerged dataframe
#the merging happens of statecode and the highest percentage value 

finalmerged=pd.merge(high,merged)

In [21]:
#reordering of columns

percent=finalmerged['percent']*100
finalmerged=finalmerged.iloc[:,[0,2]]
finalmerged['percent']=percent

In [22]:
#renaming of columns

finalmerged.columns=['state/ut', 'literacy-group', 'percentage']

In [23]:
#writing finalmerged dataframe into 

finalmerged.to_csv('literacy-india.csv',index=False)